In [1]:
import mesa
import random

In [265]:
from mesa.datacollection import DataCollector
from mesa import Agent, Model
from mesa.space import MultiGrid
import numpy as np

#Income trends
mu_c = 1000
sig_c = 200

C_num = 200 #num_of_customers
S_num = 50 #num_of_stores

#Consumer spending trends for sp
store_types = [0,1,2]
store_catgories = ['Groceries','Transport','Clothing']
store_distribution = [.5,.3,.2]
mu_sp = [50,3,200]
sig_sp = [3,4,10]
mu_base = [50,3,200]
stores = dict()
store_type = []
for i in range(S_num):
    store_type.append(np.random.choice(store_types, size = 1, p = store_distribution))




class MoneyAgentC(Agent):
   
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = round(abs(random.normalvariate(mu_c,sig_c)),2)
        self.other = ['',-1]
        self.amount = 0
        self.count = 0
        
    def salary(self):
        self.amount = round(abs(random.normalvariate(mu_c,sig_c)),2)
        self.wealth += self.amount
        self.other = ['Salary',-2]

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def give_money(self):
        cellmates = self.model.grid.get_cell_list_contents([self.pos])
        cellmates = [x for x in cellmates if type(x).__name__=='MoneyAgentSP']
        if len(cellmates) >= 1:
            other = random.choice(cellmates)
            st_type = store_type[other.unique_id][0]
            amount = round(abs(random.normalvariate(mu_sp[st_type],sig_sp[st_type])),2)
            if(amount<=self.wealth):
                other.wealth += amount
                self.wealth -= amount
                self.other = [store_catgories[st_type],other.unique_id]
                self.amount = amount 
            else:
                self.other = ['',-1]
        else:
            self.other = ['',-1]

    def step(self):
        self.count += 1
        self.move()
        if(self.count%30==0):
            self.salary()
        elif self.wealth > 0:
            self.give_money()
            
class MoneyAgentSP(Agent):
    """ An agent with fixed initial wealth."""
    def __init__(self, unique_id, model):
        super().__init__(unique_id, model)
        self.wealth = 0 
        self.other = ['a',-1]
        self.amount = 0

    def move(self):
        possible_steps = self.model.grid.get_neighborhood(
            self.pos,
            moore=True,
            include_center=False)
        new_position = random.choice(possible_steps)
        self.model.grid.move_agent(self, new_position)

    def step(self):
        return
               
class MoneyModel(Model):
    """A model with some number of agents."""
    def __init__(self, NC,NS, width, height):
        self.num_agents = NC+NS
        self.grid = MultiGrid(width, height, True)
        self.schedule = RandomActivation(self)

        # Create agents
        for i in range(NC):
            a = MoneyAgentC(i, self)
            self.schedule.add(a)
            # Add the agent to a random grid cell
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(a, (x, y))
            
        for i in range(NS):
            
            b = MoneyAgentSP(i, self)
            self.schedule.add(b)
            # Add the agent to a random grid cell
            x = random.randrange(self.grid.width)
            y = random.randrange(self.grid.height)
            self.grid.place_agent(b, (x, y))

        self.datacollector = DataCollector(
           agent_reporters={"Amount": lambda a:a.amount,"Store_type":lambda a: a.other[0], "Store_ID":lambda a:a.other[1], "Balance": lambda a : a.wealth, "Location" : lambda a:a.pos})

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

In [266]:
from mesa.time import RandomActivation
model = MoneyModel(C_num,S_num, 10, 10)
for i in range(365):
    if(i>320 and i<345):
        mu_sp = np.array(mu_base)*1.30
    else:
        mu_sp = mu_base
    model.step()

In [267]:
agent_wealth = model.datacollector.get_agent_vars_dataframe()
trans = agent_wealth[agent_wealth.Store_ID!=-1]
trans = trans[trans.Amount!=0]
trans.to_csv("Synth_data.csv")